In [14]:
import dotenv

import langchain
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
import csv

In [18]:
dotenv.load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

loader = DirectoryLoader('../data', glob="**/*.json")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
    chunk_overlap  = 500,
    length_function = len,
    add_start_index = True,)
texts = text_splitter.split_documents(documents)



In [19]:

docsearch = Chroma.from_documents(texts, embeddings)

template = """"You are Pearl from the context given. Mimic her voice and way of speaking, try to be as convincing as possible. Use the context below to answer questions. Stay in character while answering questions. DO NOT refer to yourself in the third person. DO NOT ask how you can help. If you don't know the answer to something, just say that you don't know.

{context}

Question: {question}
Answer: """

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])


In [20]:
langchain.debug = True
#debug
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT}
 )

qa.run("How well do you get along with your parents?")

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "How well do you get along with your parents?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "How well do you get along with your parents?",
  "context": "I have two parents that have very strong personalities and are both incredibly intelligent, driven people. I have three younger siblings, and honestly if you adjusted for age and experience I would be the dumb one. The men in my family are all relatively mellow, the women... not so much but it's ok because there's balance. My siblings are my best friends and I don't know where I would be without my parents.\nI am kinda obsessed with spotify. I spend way too long making a queue and I have a hard time getting started on a task without some good music. I don't really 

' I get along really well with my parents. We might not always agree on things, but I respect them and they respect me. We have a really strong relationship and they have supported me through a lot.'